In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

# Pandas Dependencies
import pandas as pd
import numpy as np
import os

# Exploring Data

In [2]:
# Path of the CSV file
file_one = "Clean_Data/hw_stations_data_clean.csv"
file_two = "Clean_Data/hw_measurements_data_clean.csv"


In [3]:
# Read CSV file into a pandas DataFrame
df_stations = pd.read_csv(file_one, dtype=object)
df_measurements = pd.read_csv(file_two, dtype=object)

In [4]:
df_stations

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84888999999998,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.3152,-157.9992,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4


In [5]:
df_measurements.head()

,New_ID,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.0,63
2,2,USC00519397,2010-01-03,0.0,74
3,3,USC00519397,2010-01-04,0.0,76
4,4,USC00519397,2010-01-06,NaN,73


# ORM Classes

In [6]:
# Use a Session to test the Station and Measurement classes
from sqlalchemy.orm import Session

engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()


In [7]:
Base = declarative_base()

#Class Stations
class Station(Base):
    __tablename__ = 'stations'

    station = Column(Text, primary_key=True,nullable=False)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
        

In [8]:
#Class Measurements
class Measurement(Base):
    __tablename__ = 'measurements'

    id = Column(Integer,primary_key=True,nullable=False)
    station = Column(Text,nullable=False)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [9]:
#Creating table schema for stations and measurements
Base.metadata.create_all(engine)

In [10]:
#List to dictionary
data_stations = df_stations.to_dict(orient='records')

In [11]:
data_measurements = df_measurements.to_dict(orient='records')

In [12]:
print(data_stations[:5])
print(data_measurements[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': '21.2716', 'longitude': '-157.8168', 'elevation': '3.0'}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': '21.4234', 'longitude': '-157.8015', 'elevation': '14.6'}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': '21.5213', 'longitude': '-157.8374', 'elevation': '7.0'}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': '21.3934', 'longitude': '-157.9751', 'elevation': '11.9'}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': '21.4992', 'longitude': '-158.0111', 'elevation': '306.6'}]
[{'New_ID': '0', 'station': 'USC00519397', 'date': '2010-01-01', 'prcp': '0.08', 'tobs': '65'}, {'New_ID': '1', 'station': 'USC00519397', 'date': '2010-01-02', 'prcp': '0.0', 'tobs': '63'}, {'New_ID': '2', 'station': 'USC00519397', 'date': '2010-01-03', 'prcp': '0.0', 'tobs': '74'}, {'New_ID': '3', 'station': 'USC00519397', 'da

In [13]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [14]:
table_stations = sqlalchemy.Table('stations', metadata, autoload=True)
table_measurements = sqlalchemy.Table('measurements', metadata, autoload=True)

In [15]:
#Cleaning tables to avoid duplicate results
conn.execute(table_stations.delete())
conn.execute(table_measurements.delete())

In [16]:
#Insert data from csv to database
conn.execute(table_stations.insert(), data_stations)
conn.execute(table_measurements.insert(), data_measurements)

In [17]:
conn.execute("select * from stations limit 5").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [18]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]